SQL databases store data in tables with a strict predened schema and follow the
ACID properties.

Atomicity - A transaction is all-or-nothing (it either completes fully or not at

Consistency – Data always remains valid and follows dened rules.

Isolation – Transactions don’t interfere with each other.


Durability – Once data is saved, it won’t be lost, even if the system crashes.


In [1]:
import redis

In [2]:
# Redis connection
redis_cache = redis.StrictRedis(host='localhost', port=6379, db=0, decode_responses=True)

In [3]:
redis_cache

<redis.client.Redis(<redis.connection.ConnectionPool(<redis.connection.Connection(host=localhost,port=6379,db=0)>)>)>